### Opinion Review 데이터 세트를 이용한 문서 Clustering

In [1]:
import pandas as pd
import glob ,os
from nltk.stem import WordNetLemmatizer
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore")

#### Opinion Review 데이터
* UCI 머신러닝 레파지토리에 있는 Opinion Review 데이터
* 51개의 텍스트 파일로 구성
* 각 파일은 Tripadvisor(호텔), Edmunds.com(자동차), Amazon.com(전자제품) 사이트에서 가져온 리뷰 문서로 구성
* 각 리뷰는 대략 100개정도의 문장으로 구성

#### 데이터를 읽어오기
* filename과 해당 file에 있는 opinion을 DataFrame으로 생성
* 위 과정을 통해 각 파일 이름(filename) 자체만으로 의견(opinion)의 텍스트가 어떠한 제품/서비스에 대한 리뷰인지 알수 있음

In [2]:
path = r'/Users/shin/Documents/git/Machinelearning-Study/NLP/OpinosisDataset1.0/topics'                     
# path로 지정한 디렉토리 밑에 있는 모든 .data 파일들의 파일명을 리스트로 취합
all_files = glob.glob(os.path.join(path, "*.data"))    
filename_list = []
opinion_text = []
# 개별 파일들의 파일명은 filename_list 리스트로 취합, 
# 개별 파일들의 파일내용은 DataFrame로딩 후 다시 string으로 변환하여 opinion_text 리스트로 취합 
for file_ in all_files:
    # 개별 파일을 읽어서 DataFrame으로 생성 
    df = pd.read_table(file_,index_col=None, header=0, encoding='latin1')
    
    # 절대경로로 주어진 file 명을 가공.
    filename_ = file_.split('/')[-1]
    filename = filename_.split('.')[0]

    #파일명 리스트와 파일내용 리스트에 파일명과 파일 내용을 추가. 
    filename_list.append(filename)
    opinion_text.append(df.to_string())

# 파일명 리스트와 파일내용 리스트를  DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,battery-life_ipod_nano_8gb,...
1,gas_mileage_toyota_camry_2007,...
2,room_holiday_inn_london,...
3,location_holiday_inn_london,...
4,staff_bestwestern_hotel_sfo,...


#### Lemmatization 구현

In [3]:
# 특수 문자나 기호를 제거하기 위한 dictionary 생성
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    # 단어를 소문자로 변환하고 특수 기호를 제거한 단어에 Lemmatization된 문장을 list로 반환
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

#### TF-IDF 형태로 피처 벡터화
* tokenizer : 토큰화를 별도의 커스텀 함수로 사용할 경우 사용
    * 위에서 정의한 LemNormalize 사용
* stop_words : 지정한 언어의 stop word로 지정된 단어는 피처에서 제외
    * english : 영어의 스톱 워드로 지정된 단어는 추출에서 제외
* ngram_range : BOW 모델의 단어 순서를 어느 정도 보강하기 위한 n_grma범위
    * (1,2) : 토큰화된 단어를 최소 1개씩 최대 2개씩 순서대로 묶어 피처로 추출
* min_df : 전체 문서에 걸쳐서 너무 낮은 빈도수를 가지는 단어 피처를 제외하기 위한 파라미터
    * 0.05 : 부동 소수점값을 가지면 전체 문서에 걸쳐 하위 5% 이하의 빈도수를 가지는 단어는 피처에서 제외
* max_df : 전체 문서에 걸쳐서 너무 높은 빈도수를 가지는 단어 피처를 제외하기 위한 파라미터
    * 0.85 : 부동 소수점값을 가지면 전체 문서에 걸쳐 상위 15% 는 피처에서 제외

In [4]:
tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english' , \
                             ngram_range=(1,2), min_df=0.05, max_df=0.85 )

#opinion_text 컬럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])
print('TfidfVector Shape : ',feature_vect.shape)

TfidfVector Shape :  (51, 4611)


####  KMeans를 활용한 Clustering 수행/평가
**scikit-learn KMeans 주요 하이퍼 파라미터**
* n_clusters : 군집화할 개수, 즉 군집 중심점(centorid)의 개수
* init : 초기에 군집 중심점의 좌표를 설정할 방식 보통 k-means++(default)를 사용
* max_iter : 최대 반복 횟수, 이 횟수 이전에 모든 데이터의 중심점 이동이 없으면 종료

#### cluster label과 file name 비교를 통한 결과 평가
* cluster_0 : 전자기기 리뷰로 cluster 형성
* cluster_1 : 자동차 리뷰로 cluster 형성
* cluster_2 : 호텔 리뷰로 cluser 형성

In [5]:
# 3개의 집합으로 군집화 
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_


# 소속 클러스터를 cluster_label 컬럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb,...,0
48,display_garmin_nuvi_255W_gps,...,0
44,fonts_amazon_kindle,...,0
41,price_amazon_kindle,...,0
40,speed_windows7,...,0
38,navigation_amazon_kindle,...,0
37,screen_netbook_1005ha,...,0
36,eyesight-issues_amazon_kindle,...,0
34,directions_garmin_nuvi_255W_gps,...,0
33,accuracy_garmin_nuvi_255W_gps,...,0


#### Cluster별 핵심 단어 추출하기
* KMeans의 cluster_centers_ 속성
    * 개별 피처가 상대적으로 각 군집의 중심점(centroid)과 얼마나 가까운지 상대 값으로 나타낸 행렬 
    * KMeans의 cluster_centers_ 속성을 이용하면 각 cluster별 핵심 단어를 찾을 수 있음

In [6]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape :',cluster_centers.shape)
print(cluster_centers)

cluster_centers shape : (3, 4611)
[[0.01005322 0.         0.         ... 0.00706287 0.         0.        ]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]]


In [7]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명들을 반환함. 
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    
    # cluster_centers array 의 값이 큰 순으로 정렬된 index 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.  
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    #개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화. 
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # cluster_centers_.argsort()[:,::-1] 로 구한 index 를 이용하여 top n 피처 단어를 구함. 
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        
        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함 
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 그리고 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames
        
    return cluster_details


In [8]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('####### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('Reviews 파일명 :',cluster_detail['filenames'][:7])
        print('==================================================')


* cluster 0(전자제품 리뷰 cluster)
    * 포터블 전자제품 리뷰 cluster인 cluster 0 에서는 'screen', 'bettery', 'life'와 같은 화면과 배터리 수명 등이 핵심 단어로 clustering
* cluster 1(자동차 리뷰 cluster):
    * 'set', 'interior', 'mileage', 'comfortable'과 같은 실내 인테리어, 좌석, 연료 효율등이 핵심 단어로 clustering
* cluster 2(호텔 리뷰 cluster):
    * 'room', 'hotel', 'service', 'location' 같은 방과 서비스 등이 핵심 단어로 clustering

In [9]:
feature_names = tfidf_vect.get_feature_names()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,\
                                  feature_names=feature_names, clusters_num=3, top_n_features=10 )
print_cluster_details(cluster_details)

####### Cluster 0
Top features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Reviews 파일명 : ['battery-life_ipod_nano_8gb', 'voice_garmin_nuvi_255W_gps', 'speed_garmin_nuvi_255W_gps', 'size_asus_netbook_1005ha', 'screen_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'satellite_garmin_nuvi_255W_gps']
####### Cluster 1
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews 파일명 : ['gas_mileage_toyota_camry_2007', 'comfort_honda_accord_2008', 'interior_toyota_camry_2007', 'transmission_toyota_camry_2007', 'seats_honda_accord_2008', 'mileage_honda_accord_2008', 'quality_toyota_camry_2007']
####### Cluster 2
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명 : ['room_holiday_inn_london', 'location_holiday_inn_london', 'staff_bestwestern_hotel_sfo', 'service_swissotel_hot